## Model Optimization

### Data Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Look at APPLICATION TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# changing cutoff for APPLICATION TYPE to up until T19
application_types_to_replace = ["T8", "T7", "T10", "T9", "T13", "T12", "T2", "T25", "T14", "T29", "T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
pd.set_option('display.max_rows', 100)
class_count = application_df["CLASSIFICATION"].value_counts().apply(pd.to_numeric)
print(class_count)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
C2190        1
C4200        1
C2600        1
C5200        1
C1370        1
C1248        1
C6100        1
C1820        1
C1900        1
C1236        1
C3700        1
C2570        1
C1580        1
C1245        1
C2500        1
C1570        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120     

In [6]:
# adopting the same cutoff value as previously
classifications_to_replace = []
for i in range(len(class_count)):
  if i > 4:
    classifications_to_replace.append(class_count.index[i])

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Look at INCOME_AMT value counts for binning
application_df["INCOME_AMT"].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [8]:
# choosing cutoff for INCOME AMOUNT to be up till anything that's above 5M
income_amt_to_replace = ["10M-50M", "5M-10M", "50M+"]

# Replace in dataframe
for inc in income_amt_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(inc,"Above 5M")

# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Above 5M           564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies = pd.get_dummies(application_df["APPLICATION_TYPE"])

class_dummies = pd.get_dummies(application_df["CLASSIFICATION"])

aff_dummies = pd.get_dummies(application_df["AFFILIATION"])

uc_dummies = pd.get_dummies(application_df["USE_CASE"])

org_dummies = pd.get_dummies(application_df["ORGANIZATION"])

income_dummies = pd.get_dummies(application_df["INCOME_AMT"])

sc_dummies = pd.get_dummies(application_df["SPECIAL_CONSIDERATIONS"])

In [10]:
# Merge converted categorial data with the main dataframe
application_df = application_df.drop(columns=["APPLICATION_TYPE", "CLASSIFICATION", "AFFILIATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"])

application_df_cleaned = application_df.merge(app_dummies,left_index=True,right_index=True).merge(class_dummies,left_index=True,right_index=True)\
.merge(aff_dummies,left_index=True,right_index=True).merge(uc_dummies,left_index=True,right_index=True).merge(org_dummies,left_index=True,right_index=True)\
.merge(income_dummies,left_index=True,right_index=True).merge(sc_dummies,left_index=True,right_index=True)

application_df_cleaned.head()

<ipython-input-10-2519e47f3771>:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Other_x'} in the result is deprecated and will raise a MergeError in a future version.
  application_df_cleaned = application_df.merge(app_dummies,left_index=True,right_index=True).merge(class_dummies,left_index=True,right_index=True)\


,STATUS,ASK_AMT,IS_SUCCESSFUL,Other_x,T19,T3,T4,T5,T6,C1000,...,Trust,0,1-9999,10000-24999,100000-499999,1M-5M,25000-99999,Above 5M,N,Y
0,1,5000,1,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,1,...,1,0,0,0,1,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df_cleaned["IS_SUCCESSFUL"].values
X = application_df_cleaned.drop(["IS_SUCCESSFUL"], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

<ipython-input-11-46dc58ea5ff5>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df_cleaned.drop(["IS_SUCCESSFUL"], 1).values


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train, Evaluate and Optimize the Model

In [13]:
# Keeping the hidden nodes for each layer the same as previously.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)

# Output layer
nn2.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid")
)

# Check the structure of the model
nn2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3120      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,581
Trainable params: 5,581
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model with epochs of 100
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5713 - accuracy: 0.7224
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5581 - accuracy: 0.7309
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5556 - accuracy: 0.7321
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5538 - accuracy: 0.7311
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5530 - accuracy: 0.7322
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5522 - accuracy: 0.7315
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5520 - accuracy: 0.7327
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5505 - accuracy: 0.7324
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5508 - accuracy: 0.7327
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5499 - accura

In [16]:
# Evaluate the second model using the test data
# Accuracy still under 75%
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5593 - accuracy: 0.7244 - 454ms/epoch - 2ms/step
Loss: 0.5593146681785583, Accuracy: 0.7244315147399902


Summary: by reducing the binning of three categorical columns did not significantly improve the model accuracy.

In [17]:
# Increase the number of hidden nodes for each layer
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 50

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)

# Output layer
nn3.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid")
)

# Check the structure of the model
nn3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               3900      
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 9,001
Trainable params: 9,001
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile and train the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn3.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5736 - accuracy: 0.7240
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5598 - accuracy: 0.7295
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5563 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5542 - accuracy: 0.7315
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5534 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5520 - accuracy: 0.7318
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5513 - accuracy: 0.7330
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5507 - accuracy: 0.7326
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5503 - accuracy: 0.7326
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5504 - accura

In [19]:
# Evaluate the third model using the test data
# Accuracy still under 75%
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5612 - accuracy: 0.7263 - 424ms/epoch - 2ms/step
Loss: 0.5611634254455566, Accuracy: 0.7262973785400391


Summary: Increasing the number of nodes of hidden layers slightly improved the accuracy of the model, but it still did not reach 75%.

In [30]:
# Optimize the model one more time with the auto optimization
def create_model(hp):
    nn4_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn4_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=100,
        step=20), activation=activation, input_dim=len(X_train[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 4)):
        nn4_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=50,
            step=10),
            activation=activation))

    nn4_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn4_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn4_model

In [21]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.8 MB/s eta 0:00:00


In [31]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=100,
    hyperband_iterations=2)

In [24]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=30,validation_data=(X_test_scaled,y_test))


Search: Running Trial #443

Value             |Best Value So Far |Hyperparameter
tanh              |relu              |activation
1                 |21                |first_units
1                 |3                 |num_layers
31                |31                |units_0
31                |1                 |units_1
21                |41                |units_2
21                |1                 |units_3
4                 |34                |tuner/epochs
0                 |12                |tuner/initial_epoch
3                 |2                 |tuner/bracket
0                 |1                 |tuner/round

Epoch 1/4
804/804 [==============================] - 5s 5ms/step - loss: 0.6343 - accuracy: 0.6510 - val_loss: 0.5888 - val_accuracy: 0.7195
Epoch 2/4
804/804 [==============================] - 4s 5ms/step - loss: 0.5814 - accuracy: 0.7258 - val_loss: 0.5830 - val_accuracy: 0.7212
Epoch 3/4
804/804 [==============================] - 6s 8ms/step - loss: 0.5787 - accuracy: 

KeyboardInterrupt: ignored

In [32]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 21,
 'num_layers': 3,
 'units_0': 31,
 'units_1': 1,
 'units_2': 41,
 'units_3': 1,
 'tuner/epochs': 34,
 'tuner/initial_epoch': 12,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0218'}

In [33]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5568 - accuracy: 0.7299 - 637ms/epoch - 2ms/step
Loss: 0.5568135380744934, Accuracy: 0.729912519454956


In [35]:
best_model.save_weights('AlphabetSoupCharity_Optimization.h5')